## Dataset Processing
#### Vanessa Chen

In [1]:
import pandas as pd
from sklearn import preprocessing

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System Edited.csv', sep=',')

In [3]:
data.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,Race/Ethnicity,LocationID,StratificationCategory1,Stratification1
0,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,32.0,30.5,33.5,7304.0,NaN,1,Total,Total
1,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,32.3,29.9,34.7,2581.0,NaN,1,Gender,Male
2,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,31.8,30.0,33.6,4723.0,NaN,1,Gender,Female
3,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,33.6,29.9,37.6,1153.0,NaN,1,Education,Less than high school
4,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,32.8,30.2,35.6,2402.0,NaN,1,Education,High school graduate


#### Removing Columns of Unnecessary or Unavailable Data

As noticed in the dataset, many parts of this dataset is not usable, repeating itself, or unnecessary for our model - assessing how the different topics variate between the different races. 

In [4]:
data.drop?

In [5]:
col_to_drop0 = ['Data_Value_Type', 'Data_Value_Alt','Data_Value_Footnote_Symbol','Data_Value_Footnote','Total',
           'Data_Value_Unit','GeoLocation']
col_to_drop1 = ['ClassID','TopicID','QuestionID','DataValueTypeID']
col_to_drop2 = ['Age(years)','Education','Gender','Income','StratificationCategoryId1','StratificationID1']
col_to_drop3 = ['LocationAbbr','Datasource','Class']

In [6]:
#data.drop(col_to_drop0, axis=1, inplace=True)
#data.drop(col_to_drop1, axis=1, inplace=True)
#data.drop(col_to_drop2, axis=1, inplace=True)
#data.drop(col_to_drop3, axis=1, inplace=True)
data.drop('LocationID', axis=1, inplace=True)


In [12]:
newData = data[data['StratificationCategory1'] == 'Race/Ethnicity']

In [14]:
newData['StratificationCategory1'].value_counts()

Race/Ethnicity    15256
Name: StratificationCategory1, dtype: int64

In [15]:
#data.drop(data.StratificationCategory1 != 'Race/Ethnicity', axis=1, inplace=True)
#this doesn't seem to work...shouldn't it create a new dataframe without any rows where the StratificationCategory1 
#column does not contain 'Race/Ethnicity'?
df2 = data[data.StratificationCategory1 != 'Race/Ethnicity']

In [16]:
df2.StratificationCategory1.value_counts()

Income         13349
Age (years)    11438
Education       7628
Gender          3814
Total           1907
Name: StratificationCategory1, dtype: int64

In [12]:
#data.drop('StratifiactionCategory1' = 'Gender', inplace=True)
#I'll just drop the null values in the 'Race/Ethnicity column'

data['Race/Ethnicity'].dropna() #why didn't this work either? 

Series([], Name: Race/Ethnicity, dtype: object)

In [11]:
#data_new = data[data.StratificationCategory1 != 'Race/Ethnicity']
data.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,Race/Ethnicity,StratificationCategory1,Stratification1
0,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,32.0,30.5,33.5,7304.0,NaN,Total,Total
1,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,32.3,29.9,34.7,2581.0,NaN,Gender,Male
2,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,31.8,30.0,33.6,4723.0,NaN,Gender,Female
3,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,33.6,29.9,37.6,1153.0,NaN,Education,Less than high school
4,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,32.8,30.2,35.6,2402.0,NaN,Education,High school graduate


In [29]:
data.drop?

In [21]:
# Converting values in the school column to text
# We are going to define a function that takes a single value and apply it to all the values
def convert_Stratification1(row):
    if row == 'Non-Hispanic White':
        return 0
    elif row == 'Non-Hispanic Black':
        return 1
    elif row == 'Hispanic':
        return 2
    elif row == 'Asian':
        return 3
    elif row == 'Hawaiian/Pacific Islander':
        return 4
    elif row == 'American Indian/Alaska Native':
        return 5
    elif row == '2 or more races':
        return 6
    elif row == 'Other':
        return 7
    else:
        return None

In [24]:
converted_data_Stratification = []
converted_data_Stratification = data['Stratification1'].apply(convert_Stratification1) #takes in a function that is applied to every row, 
#and applied to that column, but instead of doing it one by one, it does all the rows at the same time

In [25]:
print(converted_data_Stratification)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
28       None
29       None
30       None
31       None
32       None
33       None
34       None
35       None
36       None
37       None
         ... 
53346    None
53347    None
53348    None
53349    None
53350    None
53351    None
53352    None
53353    None
53354    None
53355    None
53356    None
53365    None
53366    None
53367    None
53368    None
53369    None
53370    None
53371    None
53372    None
53373    None
53374    None
53375    None
53376    None
53377    None
53378    None
53379    None
53380    None
53381    None
53382    None
53383    None
Name: Stratification1, Length: 38136, dtype: object


#### Converting Categorical Values to Numerical Ones

Looking at the data above, we want to convert a number of the columns from categorical to numerical. Most machine learning models deal with numbers and don't know how to model data that is in text form. As a result we need to learn how to do things such as e.g., convert the values in the `school` column to numbers.

#### First, let's see what values there are in the `school` column

In [44]:
# This shows a list of unique values and how many times they appear
new_data['school'].value_counts()

KeyError: 'school'

In [18]:
# Converting values in the school column to text
# We are going to define a function that takes a single value and apply it to all the values
def convert_school(row):
    if row == 'GP':
        return 0
    elif row == 'MS':
        return 1
    else:
        return None

In [20]:
# Here's a slow way of using the above function
%time #this can show you the time it took
converted_data = []

for row in data['school']:
    new_value = convert_school(row)
    converted_data.append(new_value)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


KeyError: 'school'

In [ ]:
print(converted_data)

In [19]:
%time
converted_data = data['school'].apply(convert_school) #takes in a function that is applied to every row, 
#and applied to that column, but instead of doing it one by one, it does all the rows at the same time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


KeyError: 'school'

#### Using sklearn's built-in preprocessing module, we can do the same thing

In [ ]:
enc = preprocessing.LabelEncoder() #this is prebuilt and helps us 

In [ ]:
transformed = enc.fit_transform(data['school'])  

In [ ]:
transformed

#### Dealing with Null values

To show you how to deal with null values, I'm going to make some simulated data of students.

In [14]:
grades = np.random.choice(range(1, 13), 100) # chooses 100 random numbers between 1 - 12
num_friends_or_none = list(range(0, 20)) + [None] * 5
num_friends = np.random.choice(num_friends_or_none, 100)
new_data = pd.DataFrame(data={'Grade': grades, '# Friends': num_friends})

In [15]:
new_data.head(n=20)

,# Friends,Grade
0,1,7
1,None,2
2,8,5
3,18,8
4,None,9
5,17,3
6,8,8
7,None,10
8,19,1
9,18,10


#### One way to deal with null values is to drop them

In [42]:
new_data['# Friends'].dropna()#returns a dataset with all the null values gone - won't actually replace the original data

0      4
2     11
4      9
5      7
6     12
8      0
9     16
10    14
11    14
12     0
13     3
15     9
16     2
18    17
19    15
20    16
21     9
22     7
23    16
24     0
26     7
28    19
29     5
32    19
33    18
35    15
36     2
37     6
38    17
39     7
      ..
61    13
62     6
63     1
64     0
65     7
66     9
67    16
68    16
69     6
70     6
71    16
72     3
73     3
74    12
75    18
76     9
78    11
79     1
80    16
82    17
84     8
85    12
90    15
91     7
92    16
93    14
95    17
96     3
97    19
98    17
Name: # Friends, Length: 77, dtype: object

In [17]:
dropNas = pd.DataFrame()
dropNas['# Friends'] = new_data['# Friends'].dropna()
dropNas

,# Friends
0,1
2,8
3,18
5,17
6,8
8,19
9,18
11,8
12,14
13,7


In [ ]:
average_friends = new_data['# Friends'].mean()
new_data['# Friends'].fillna(average_friends) #replacing that column with a number of your choosing (fillna() method)

In [ ]:
new_data['# Friends'] = new_data['# Friends'].fillna(average_friends)

#### Now let's learn how to standardize data
By that I mean to transform our data so that it has a mean of 0 and a standard deviation of 1

In [ ]:
from sklearn.preprocessing import StandardScaler #so that it's easier to look at your data - so that your data is centered at 0
#helps remove the issue of having super high ranges

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit_transform(dropNas['# Friends'].values.reshape(-1,1)) #-1 means if you don't know the number of rows, so 
#you ask the computer to find the most optimal